In [1]:
# 

In [2]:
import json
import pandas as pd
from rapidfuzz import fuzz

from sklearn.model_selection import train_test_split

In [3]:
input_file_path = '/home/hmx/Downloads/RumourVerification/data/English_test.json'

# Path to the corrected JSON file
corrected_file_path = '/home/hmx/Downloads/RumourVerification/data/Corrected_English_test.json'
output_file_path = '/home/hmx/Downloads/RumourVerification/data/Updated_English_test.json'

output_file_with_pred_path = '/home/hmx/Downloads/RumourVerification/data/Updated_English_test_with_predictions.json'


# train data
train_data_file_path = '/home/hmx/Downloads/RumourVerification/data/English_train.json'

In [4]:
import json


# Step 1: Read the entire file content line by line
json_objects = []
with open(input_file_path, 'r', encoding='utf-8') as file:
    for line in file:
        if line.strip():  # Check if the line is not empty
            json_objects.append(json.loads(line))

# Step 2: Write the list of JSON objects to a new file
with open(corrected_file_path, 'w', encoding='utf-8') as file:
    json.dump(json_objects, file, ensure_ascii=False, indent=4)

print(f"Corrected JSON file has been saved to {corrected_file_path}")

Corrected JSON file has been saved to /home/hmx/Downloads/RumourVerification/data/Corrected_English_test.json


In [5]:
# Step 1: Read the corrected JSON data
with open(corrected_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Step 2: Calculate similarity and update the timeline
for item in data:
    rumor = item.get('rumor', '')
    timeline = item.get('timeline', [])
    
    # Calculate similarity for each element in the timeline
    for i in range(len(timeline)):
        similarity = fuzz.ratio(rumor, timeline[i][2])  # Assuming the text to compare is in the third position
        timeline[i].append(similarity)
    
    # Sort the timeline based on similarity values
    timeline.sort(key=lambda x: x[-1], reverse=True)

# Step 3: Save the updated JSON data to a new file
with open(output_file_path, 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

print(f"Updated JSON file has been saved to {output_file_path}")

Updated JSON file has been saved to /home/hmx/Downloads/RumourVerification/data/Updated_English_test.json


In [6]:
# Train-Prediction Phase

def parse_json_file_to_df(file_path):
    parsed_data = {
        'ID': [],
        'Rumor': [],
        'Label': [],
        'Timeline_URLs': [],
        'Timeline_Tweet_IDs': [],
        'Timeline_Contents': [],
        'Evidence_URLs': [],
        'Evidence_Tweet_IDs': [],
        'Evidence_Contents': []
    }
    
    with open(file_path, 'r') as file:
        for line in file:
            try:
                item = json.loads(line)
                
                parsed_data['ID'].append(item.get('id'))
                parsed_data['Rumor'].append(item.get('rumor'))
                parsed_data['Label'].append(item.get('label'))
                
                timeline_urls = []
                timeline_tweet_ids = []
                timeline_contents = []
                for timeline_entry in item.get('timeline', []):
                    timeline_urls.append(timeline_entry[0])
                    timeline_tweet_ids.append(timeline_entry[1])
                    timeline_contents.append(timeline_entry[2])
                parsed_data['Timeline_URLs'].append(timeline_urls)
                parsed_data['Timeline_Tweet_IDs'].append(timeline_tweet_ids)
                parsed_data['Timeline_Contents'].append(timeline_contents)
                
                evidence_urls = []
                evidence_tweet_ids = []
                evidence_contents = []
                for evidence_entry in item.get('evidence', []):
                    evidence_urls.append(evidence_entry[0])
                    evidence_tweet_ids.append(evidence_entry[1])
                    evidence_contents.append(evidence_entry[2])
                parsed_data['Evidence_URLs'].append(evidence_urls)
                parsed_data['Evidence_Tweet_IDs'].append(evidence_tweet_ids)
                parsed_data['Evidence_Contents'].append(evidence_contents)
                
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON: {e}")
                continue
    
    df = pd.DataFrame(parsed_data)
    return df

In [7]:
df = parse_json_file_to_df(train_data_file_path)
df

,ID,Rumor,Label,Timeline_URLs,Timeline_Tweet_IDs,Timeline_Contents,Evidence_URLs,Evidence_Tweet_IDs,Evidence_Contents
0,AuRED_014,“#Urgent / Ramallah Ministry of Health spokesm...,REFUTES,"[https://twitter.com/ibrahimmilhim, https://tw...","[1357270458756960257, 1357270456915685377, 135...","[""Qalqilya (5), Bethlehem (10), Nablus (24), J...","[https://twitter.com/palestine_moh, https://tw...","[1357211717479116800, 1356683687870488576, 135...",[Pictures from the launch of the vaccination c...
1,AuRED_037,Macron to Sky News: After my visit to Mrs. Fai...,REFUTES,"[https://twitter.com/skynewsarabia, https://tw...","[1302029928867729411, 1302028670446444544, 130...",[#Iraq.. Record infections with #Corona and ho...,"[https://twitter.com/salmanonline, https://twi...","[1301079467889131522, 1301067546200858626, 130...",[RT @HalaHomsi: Macron and Mrs. Fairouz: This ...
2,AuRED_085,Saudi Arabia evacuated 10 students from China ...,REFUTES,"[https://twitter.com/YSUCORG, https://twitter....","[1226482043007066113, 1226481466348986369, 122...",[RT @Yemen_PM: The Yemeni Embassy in #China an...,"[https://twitter.com/YSUCORG, https://twitter....","[1226482043007066113, 1225345358961762304, 122...",[RT @Yemen_PM: The Yemeni Embassy in #China an...
3,AuRED_089,"The Corona epidemic has reached the Emirates, ...",REFUTES,"[https://twitter.com/WHOEMRO, https://twitter....","[1223614769195900928, 1223608938136047616, 122...",[s. Is it safe to receive letters or packages ...,"[https://twitter.com/WHOEMRO, https://twitter....","[1222506828694794240, 1223649306667778049, 122...",[Today @WHO confirmed the emergence of the fir...
4,AuRED_135,The official spokesman for the Football Associ...,REFUTES,"[https://twitter.com/AlAhlyTV, https://twitter...","[1586138070692548610, 1586135058083381248, 158...",[Special- Dr. Khaled Al-Jawadi: Marcel is a di...,"[https://twitter.com/AlAhlyTV, https://twitter...","[1585012773125120000, 1585011015032315904]",[Islam Al-Shater “warns” against fabricating n...
...,...,...,...,...,...,...,...,...,...
91,AuRED_055,The Moroccan referee apologizes to Al-Ahly pla...,NOT ENOUGH INFO,"[https://twitter.com/AlAhly, https://twitter.c...","[1369774548585963524, 1369773418191740936, 136...","[ISSUE: couldn't translate, ISSUE: couldn't tr...",[],[],[]
92,AuRED_023,Good news for Al-Ahly fans.. BeIN Sports choos...,NOT ENOUGH INFO,"[https://twitter.com/derradjihafid, https://tw...","[1356265561634844674, 1356252307885875202, 135...","[""Leave him alone... National coach Jamal Belm...",[],[],[]
93,AuRED_016,Egypt does not want to give a vaccine to its c...,NOT ENOUGH INFO,"[https://twitter.com/mohpegypt, https://twitte...","[1391904405012746240, 1391875695807062019, 139...",[Health: The number of recoveries from those i...,[],[],[]
94,AuRED_077,"Egyptian Dr. Sir (Magdy Yacoub), the greatest ...",NOT ENOUGH INFO,"[https://twitter.com/FCDOArabic, https://twitt...","[1345122644589486080, 1345110666932342785, 134...",[RT @UKinSudan: Her Majesty Queen Elizabeth se...,[],[],[]


In [8]:

train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

In [9]:
from stormtrooper import SetFitFewShotClassifier

model = SetFitFewShotClassifier(model_name="sentence-transformers/all-mpnet-base-v2", device='cuda').fit(df['Rumor'].astype('str').tolist(), df['Label'].astype('str').tolist())

test_preds = model.predict(test_df['Rumor'].astype('str').tolist())

# calculate accuracy and f1 score base on the prediction and  true labels (test_df['Label'].astype('str').tolist())) 
from sklearn.metrics import accuracy_score, f1_score
accuracy = accuracy_score(test_df['Label'].astype('str').tolist(), test_preds, normalize=False)
f1 = f1_score(test_df['Label'].astype('str').tolist(), test_preds, average='weighted')


print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

/home/hmx/miniconda3/envs/rumour_verification/lib/python3.9/site-packages/thefuzz/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
/home/hmx/miniconda3/envs/rumour_verification/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/hmx/miniconda3/envs/rumour_verification/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights

{'embedding_loss': 0.3125, 'learning_rate': 8.333333333333333e-07, 'epoch': 0.0}


{'embedding_loss': 0.279, 'learning_rate': 1.7592592592592595e-05, 'epoch': 0.21}


{'embedding_loss': 0.0883, 'learning_rate': 1.2962962962962964e-05, 'epoch': 0.42}


{'embedding_loss': 0.0258, 'learning_rate': 8.333333333333334e-06, 'epoch': 0.62}


{'embedding_loss': 0.0017, 'learning_rate': 3.7037037037037037e-06, 'epoch': 0.83}



100%|██████████| 240/240 [02:21<00:00,  1.70it/s]


{'train_runtime': 141.7662, 'train_samples_per_second': 27.087, 'train_steps_per_second': 1.693, 'epoch': 1.0}


AttributeError: 'Series' object has no attribute 'tolistcorrected_file_path'

In [10]:
# Load the corrected JSON data
with open(output_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)
    
# Prepare the test data (rumors) from the JSON file
test_rumors = [item['rumor'] for item in data]

    # Add predictions to the JSON data
predictions = model.predict(test_rumors)
for item, prediction in zip(data, predictions):
    item['prediction'] = prediction

# Save the updated JSON data with predictions to a new file
with open(output_file_with_pred_path, 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

print(f"Updated JSON file with predictions has been saved to {output_file_path}")

Updated JSON file with predictions has been saved to /home/hmx/Downloads/RumourVerification/data/Updated_English_test.json
